In [1]:
import re
import numpy as np
import tensorflow as tf

## 데이터 읽어오기

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.']


## 데이터 정제

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기를 권합니다.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    
#     temp = re.subn(r"[A-Za-z]", " ", "[appleAPPLE34] gogo")[0]
#     temp = re.sub(r'[" "]+', " ", temp)
#     temp = re.sub(r"[^a-zA-Z?.!,¿]+", " ", temp) 
#     temp = temp.strip()
    
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=17000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
#     tensor = np.delete(tensor, slice(13, -1), axis=1)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  903 ...    0    0    0]
 [   2  122 2584 ...    0    0    0]
 [   2    6  304 ...    0    0    0]
 ...
 [ 673   27    6 ...    6  189    3]
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f046004b6d0>


In [6]:
tensor.shape

(175749, 15)

In [7]:
len(tokenizer.index_word)

27592

In [8]:
'chorus' in tokenizer.index_word.values()

True

In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


## 평가 데이터셋 분리
- 단어장의 크기를 12,000이상으로...? num_words를 말하는 건가

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train, test = train_test_split(tensor, test_size=0.2)

In [12]:
train.shape, test.shape

((140599, 15), (35150, 15))

In [13]:
enc_train = train[:, :-1]
dec_train = train[:, 1:]
enc_val = test[:, :-1]
dec_val = test[:, 1:]

In [14]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [15]:
print(enc_train[0])
print(dec_train[0])

[   2   13 1735  188    9 5406    3    0    0    0    0    0    0    0]
[  13 1735  188    9 5406    3    0    0    0    0    0    0    0    0]


In [16]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [17]:
valid_ds = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE, drop_remainder=True)
valid_ds

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 학습

In [18]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.linear = tf.keras.layers.Dense(vocab_size)
        
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.batch_norm_1(out)
        out = self.rnn_2(out)
        out = self.batch_norm_2(out)
        out = self.linear(out)
        
        return out

# 조절 ---------
embedding_size = 2400
hidden_size = 3000
# --------------

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [19]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 17001), dtype=float32, numpy=
array([[[-9.01459192e-04, -1.72119966e-04, -7.69188511e-04, ...,
          1.90191000e-04, -3.36569683e-05, -3.69233567e-05],
        [-1.17572118e-03, -5.82913170e-04, -1.27777213e-03, ...,
          1.85036861e-05, -2.30248581e-04,  3.80951184e-04],
        [-2.00513168e-03, -3.33200063e-04, -1.22158765e-03, ...,
         -4.48822946e-04, -3.46166693e-04,  3.59070546e-04],
        ...,
        [-1.13080686e-03,  6.36165147e-04, -6.28359802e-03, ...,
          1.61068724e-03,  5.55038871e-03, -4.26885858e-03],
        [-1.38767797e-03,  1.50411768e-04, -7.13498332e-03, ...,
          1.32139202e-03,  5.92282601e-03, -5.07868826e-03],
        [-1.63325644e-03, -3.79786128e-04, -7.81564042e-03, ...,
          9.67674598e-04,  6.21154113e-03, -5.87901333e-03]],

       [[-9.01459192e-04, -1.72119966e-04, -7.69188511e-04, ...,
          1.90191000e-04, -3.36569683e-05, -3.69233567e-05],
        [-1.44788087e-03,  4.50756779e-04, -4

In [20]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  40802400  
_________________________________________________________________
lstm (LSTM)                  multiple                  64812000  
_________________________________________________________________
batch_normalization (BatchNo multiple                  12000     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  72012000  
_________________________________________________________________
batch_normalization_1 (Batch multiple                  12000     
_________________________________________________________________
dense (Dense)                multiple                  51020001  
Total params: 228,670,401
Trainable params: 228,658,401
Non-trainable params: 12,000
_________________________________

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=valid_ds, epochs=10)

Epoch 1/10
549/549 [==============================] - 608s 1s/step - loss: 3.2213 - val_loss: 4.7352
Epoch 2/10
549/549 [==============================] - 610s 1s/step - loss: 2.6056 - val_loss: 2.6256
Epoch 3/10
549/549 [==============================] - 719s 1s/step - loss: 2.2506 - val_loss: 2.4785
Epoch 4/10
549/549 [==============================] - 615s 1s/step - loss: 1.9572 - val_loss: 2.3703
Epoch 5/10
549/549 [==============================] - 614s 1s/step - loss: 1.7065 - val_loss: 2.3059
Epoch 6/10
549/549 [==============================] - 615s 1s/step - loss: 1.4972 - val_loss: 2.2645
Epoch 7/10
549/549 [==============================] - 615s 1s/step - loss: 1.3268 - val_loss: 2.2394
Epoch 8/10
549/549 [==============================] - 620s 1s/step - loss: 1.1937 - val_loss: 2.2278
Epoch 9/10
549/549 [==============================] - 615s 1s/step - loss: 1.0941 - val_loss: 2.2520
Epoch 10/10
549/549 [==============================] - 615s 1s/step - loss: 1.0257 - val_lo

In [22]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 어우

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love me like you do <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , <end> '

## 가사 생성

- 가사 데이터를 학습
- (LSTM - BatchNormalization) X 2

###  목표 Loss (2.2 이하, epoch = 10 고정)

---

1. 20.09.09 : **2.2278**

   - 하이퍼 파라미터

     ```python
     num_words = 17000
     BATCH_SIZE = 256
     embedding_size = 2400
     hidden_size = 3000
     learning_rate = 0.0001
     ```

   - 파라미터수 (2억 2천8백만...)
   
       ![image](https://user-images.githubusercontent.com/48716219/92634317-11fbe000-f30f-11ea-9b96-a23c43369498.png)

   - 학습 진행

     ![image](https://user-images.githubusercontent.com/48716219/92634558-70c15980-f30f-11ea-814b-398f2cdbec93.png)